# How to systematically build regular expression patterns

In this notebook, we show how to systematically develop regular expression patterns that can be used for information extraction.

Regular expression patterns developed for large-scale information extraction typically grow lengthy and complex. 
In such situations, it can be difficult to keep track, which parts of the pattern were designed for which purpose, let alone to fully ensure that each sub expression matches all of its intended targets.

EstNLTK provides class `RegexElement`, which wraps around the [regex library](https://pypi.org/project/regex/) and simplifies 
documenting and testing regex patterns. 
It is possible to add tests for positive, negative and partial matches, and to automatically test patterns. 
`RegexElement` subclasses also add a way to construct regular expressions in an hierarchical manner together with test synthesis, which automatically combines existing tests of sub-expression.

For the sake of simplicity, we use a toy example in this tutorial. 
We show how to develop a regex for extracting ingredient information from food recipes, such as:

In [1]:
example_ingredients = """
1 pakk tordipulbrit 
4 tk muna 
400 g kohupiimapastat
4 sl suhkrut
200 g hapukoort
100 g võid
"""

In [2]:
from estnltk.taggers.system.rule_taggers.regex_library.regex_element import RegexElement

# Create pattern
KOGUS = RegexElement('([0-9]+[.,])?[0-9]+', group_name='quantity', 
                   description='Captures integer and decimal quantities.')
# Add tests
KOGUS.full_match('1')
KOGUS.full_match('100')
KOGUS.full_match('1,5')
KOGUS.full_match('2.5')
KOGUS.no_match('x')
KOGUS.no_match('N/A')
KOGUS.no_match('teadmata arv')
KOGUS.partial_match('<=3', '3')
KOGUS.partial_match('~2', '2')

# Create pattern
# (Note: there's a better way to do a string choice, read about the StringList below)
YHIK = RegexElement('(supilusika[ts]|teelusika[ts]|pakki?|grammi?|tk|sl|g)', group_name='unit',
                     description='Captures 5 types of food units, including 3 unit abbreviations.')
# Add tests
YHIK.full_match('supilusikat')
YHIK.full_match('teelusikas')
YHIK.full_match('tk')
YHIK.full_match('pakk')
YHIK.full_match('gramm')
YHIK.full_match('sl')
YHIK.full_match('g')
YHIK.no_match('tonni')
YHIK.partial_match('100g', 'g')
YHIK.partial_match('2tk', 'tk')

# Create a pattern combining other patterns
KOOSTISOSA = RegexElement(fr'{KOGUS}\s*{YHIK}\s+(?P<ingredient>[a-zöäüõšž]+)', 
                          description='Captures ingredients together with quantities and units.')
# Add tests
KOOSTISOSA.full_match('1 supilusikas suhkrut')
KOOSTISOSA.full_match('2.5 sl võid')
KOOSTISOSA.full_match('4 teelusikat siirupit')
KOOSTISOSA.full_match('300g vahukoort')
KOOSTISOSA.no_match('10 tonni telliskive')
KOOSTISOSA.partial_match('100g vahukomme ka', {'quantity':'100', 'unit':'g', 'ingredient':'vahukomme'})

`RegexElement` parameters:
* `pattern` -- the regular expression pattern. For details about constructing regular expressions in Python, see https://docs.python.org/3/howto/regex.html , https://docs.python.org/3/library/re.html and https://pypi.org/project/regex/ ;
* `group_name` -- (Optional) adds a named capture group around the regex. The `group_name` appears as the top level group in string representation of this expression (`str(regex_element)`), but the regular expression `pattern` itself must not contain the `group_name`. If `group_name` is not provided, then the regex is placed inside a non-capture group;
* `description` -- (Optional) used in the display and should concisely describe the intent behind the pattern. Additional information about the intent can be specified through examples which are also used in the display (See "Examples for displaying" below).

Methods for adding tests:
* `full_match(example:str, description: str = None)` -- adds a positive example. The pattern must match with the whole example;
* `no_match(example:str, description: str = None)` -- adds a negative example. The pattern must not be found inside the example, that is, it cannot match even with a substring of the example;
* `partial_match(text: str, target: Union[str, Dict[str, str]], description: str = None)` adds an extraction example which requires the pattern to match the substring `target` inside `text`. Alternatively, `target` can also be a dictionary specifying exact matches required from capture groups of the `pattern` (mappings `group_name` -> `target_str`) when applied on `text`.

All methods allow to add more detailed descriptions of examples via parameter `description`.

### Testing patterns

Once you've defined a pattern and added tests, you can use the method `test()` to run all tests:

In [3]:
KOGUS.test()
YHIK.test()
KOOSTISOSA.test()

The method runs silent if all tests are passed. 
However, if any of the tests should fail, an `AssertionError` will be thrown, informing about the details of the test case on which the pattern fails. 

In Notebook, you can get a quick overview about the pattern and its testing status when you display a `RegexElement` object:

In [4]:
KOGUS

Test group,passed,failed
positive examples,4,0
negative examples,3,0
extraction tests,2,0


In [5]:
YHIK

Test group,passed,failed
positive examples,7,0
negative examples,1,0
extraction tests,2,0


In [6]:
KOOSTISOSA

Test group,passed,failed
positive examples,4,0
negative examples,1,0
extraction tests,1,0


To get **exact testing results**, use methods `evaluate_positive_examples()`, `evaluate_negative_examples()` and `evaluate_extraction_examples()`. The results will be returned in a `DataFrame`:

In [7]:
YHIK.evaluate_positive_examples()

,Example,Status
0,supilusikat,+
1,teelusikas,+
2,tk,+
3,pakk,+
4,gramm,+
5,sl,+
6,g,+


In [8]:
YHIK.evaluate_negative_examples()

,Example,Status
0,tonni,+


In [9]:
YHIK.evaluate_extraction_examples()

,Example,Status
0,100g,+
1,2tk,+


Status `+` indicates a passing test and status `F` indicates a failing one.

### Examples for displaying

You can add some of the positive examples as _display examples_, which will be show when the object is rendered in Notebook.

For instance, let's redefine KOOSTISOSA with first 3 `full_match` tests as _display examples_:

In [10]:
KOOSTISOSA = RegexElement(fr'{KOGUS}\s*{YHIK}\s+(?P<ingredient>[a-zöäüõšž]+)', 
                          description='Captures ingredients together with quantities and units.')
# Add tests & examples
KOOSTISOSA.example('1 supilusikas suhkrut')
KOOSTISOSA.example('2.5 sl võid')
KOOSTISOSA.example('4 teelusikat siirupit')
KOOSTISOSA.full_match('300g vahukoort')
KOOSTISOSA.no_match('10 tonni telliskive')
KOOSTISOSA.partial_match('100g vahukomme ka', {'quantity':'100', 'unit':'g', 'ingredient':'vahukomme'})

In [11]:
# Browse the pattern
KOOSTISOSA

(?:(?P<quantity>([0-9]+[.,])?[0-9]+)\s*(?P<unit>(supilusika[ts]|teelusika[ts]|pakki?|grammi?|tk|sl|g))\s+(?P<ingredient>[a-zöäüõšž]+))

Note that there are 4 positive examples: all _display examples_ were also included in positive (`full_match`) examples.

### Choice groups

Regex choice groups, such as `'(supilusika[ts]|teelusika[ts]|pakki?|grammi?|tk|sl|g)'`, allow to specify multiple alternative patterns to be searched for. 

Regex choice groups can contain sub-expressions with overlapping targets. 
For instance, _YHIK_ (unit) in the previous example contains choice sub-expressions `grammi?` and `g`, which both match with the string `'gramm'`.
However, the extent of the match depends on the order of the sub-expressions in the group: the maximal extent is achieved only if sub-expressions capturing longest strings come first. 
The result of a wrong ordering is an incomplete match, e.g. `(g|grammi?)` matches only `'g'` inside the string `'gramm'`.

If regex choice groups grow large and complex, it can be difficult to achieve a correct ordering. 
A rigorous work of pattern development and testing is required. 
However, for subsets of patterns satisfying specific constraints, correct ordering of the sub-expressions can be automatically guaranteed.

#### StringList

Use `StringList` to make a choice group from (a large number of) strings. It will guarantee that the resulting regular expression matches even the longest string in the list; it will also detect and remove duplicates, escape all the meta symbols (such as `.` or `+`) and convert the pattern to a case insensitive format (if requested).

For instance, let's redefine YHIK as a `StringList`, which has more units / unit abbreviations and which ignores case while matching the strings:

In [12]:
from estnltk.taggers.system.rule_taggers.regex_library.string_list import StringList

# Create pattern
YHIK = StringList(['supilusikas', 'supilusikat', 'sl', 
                   'teelusikas', 'teelusikat', 'tl',
                   'pakk', 'pakki', 'pk', 
                   'tükk', 'tükki', 'tk', 
                   'gramm', 'grammi', 'g'], group_name='unit',
                   description='Captures 5 types of food units (incl abbreviations).',
                   ignore_case=True)
# Add tests
YHIK.full_match('SUPILUSIKAS')
YHIK.full_match('SupiLusiKaT')
YHIK.full_match('teelusikas')
YHIK.full_match('tk')
YHIK.full_match('TK')
YHIK.full_match('pakk')
YHIK.full_match('PAKK')
YHIK.full_match('gramm')
YHIK.full_match('sl')
YHIK.full_match('g')
YHIK.no_match('tonni')
YHIK.no_match('puuda')
YHIK.partial_match('100g', 'g')
YHIK.partial_match('2tk', 'tk')
YHIK.partial_match('2Tk', 'Tk')

In [13]:
YHIK

Test group,passed,failed
positive examples,10,0
negative examples,2,0
extraction tests,3,0


You can also generalize characters appearing in strings, e.g. replace all spaces with a more general pattern `r'\s+'`. 
Parameter `replacements` allows to defined a dictionary of character to regex replacements that is applied to all strings. 
For instance:

In [14]:
TOIDUAINED = StringList(['muna', 'tordipulbrit', 'kohupiimapastat', 'suhkrut', 'hapukoort', 'võid', 'siirupit', 
                         'valget šokolaadi', 'tumedat šokolaadi', 'maasika jäätist', 'vahukoort', 'vahukomme'], 
                         group_name='ingredient',
                         description='Captures ingredient names.',
                         replacements={' ' : r'\s+'}, 
                         ignore_case=True)
# Add tests & examples
TOIDUAINED.example('suhkrut')
TOIDUAINED.example('võid')
TOIDUAINED.example('siirupit')
TOIDUAINED.full_match('vahukoort')
TOIDUAINED.full_match('tumedat   šokolaadi')
TOIDUAINED.full_match('valget  šokolaadi')
TOIDUAINED.no_match('telliskive')
TOIDUAINED

(?P<ingredient>[Tt][Uu][Mm][Ee][Dd][Aa][Tt](?:\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Vv][Aa][Ll][Gg][Ee][Tt](?:\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Kk][Oo][Hh][Uu][Pp][Ii][Ii][Mm][Aa][Pp][Aa][Ss][Tt][Aa][Tt]|[Mm][Aa][Aa][Ss][Ii][Kk][Aa](?:\s+)[Jj][Ää][Ää][Tt][Ii][Ss][Tt]|[Tt][Oo][Rr][Dd][Ii][Pp][Uu][Ll][Bb][Rr][Ii][Tt]|[Hh][Aa][Pp][Uu][Kk][Oo][Oo][Rr][Tt]|[Vv][Aa][Hh][Uu][Kk][Oo][Mm][Mm][Ee]|[Vv][Aa][Hh][Uu][Kk][Oo][Oo][Rr][Tt]|[Ss][Ii][Ii][Rr][Uu][Pp][Ii][Tt]|[Ss][Uu][Hh][Kk][Rr][Uu][Tt]|[Mm][Uu][Nn][Aa]|[Vv][Õõ][Ii][Dd])

Note that the left hand of the replacement rule can be only **a single character that is interpreted as a plain character**, not a regex meta character. 
The right-hand side of the rule is **a regex string**, in which meta characters (such as `.` or `+`) need to be properly escaped.

#### StringList import

You can also import StringList-s from CSV or TXT files. Use the function `StringList.from_file()`, which has the following signature:

```python
StringList.from_file(file: Union[str, List[str]],
                     column: str = None,
                     group_name: str = None,
                     description: str = None,
                     replacements: Dict[str, str] = None,
                     ignore_case: bool = False,
                     ...) -> StringList:
```

* `file` can be either a single file path or a list of file paths. Each file name must have `.txt` or `.csv` extension, which is used to determine file type;
* `column` is used to extract strings from specific column in `.csv` files; if not provided, then strings will be extracted frm the first column by default. In `.txt` files, each row will extracted as an element of the string list;
* Parameters `group_name`, `description`, `replacements`, `ignore_case` are passed on to the `StringList` upon initialization; note that these values need to be set beforehand, as they cannot be changed after the `StringList` has been initialized;
* `...` -- additional arguments can be used to control the csv parsing with `pandas.read_csv` function;

#### StringList export

You can export strings of a StringList into a CSV via the method `to_csv()`:

```python
my_stringlist.to_csv(file: str, 
                     column: str = 'string_list')
```

Note that only plain text strings will be saved, without any post-processing (no sorting, no removal of duplicates nor applying replacements).

#### ChoiceGroup

`ChoiceGroup` allows to systematically construct and test complex `RegexElement` choice groups. Currently, it provides the following functionalities:

* If all choice group elements are `StringList`-s, then it automatically constructs the expression in a way that all strings are properly ordered (matching with the longest string is guaranteed) and `ignore_case` settings of all `StringList`-s will be preserved. Note that all `StringList`-s must have the same `replacements`.
* All tests of choice group sub-expressions can be automatically carried over to tests of the `ChoiceGroup`, so it can be tested whether the order of sub-expressions is correct / satisfies tests. Note, however, that this is not a default behaviour, but needs to be switched on by special flags.

Let's rewrite KOGUS (quantity) pattern in a way that it allows ranges of quantities in addition to simple quantities:

In [15]:
from estnltk.taggers.system.rule_taggers.regex_library.choice_group import ChoiceGroup

# Create pattern
ARV = RegexElement('([0-9]+[.,])?[0-9]+', description='Captures integer and decimal quantities.')
# Add tests
ARV.full_match('1')
ARV.full_match('100')
ARV.full_match('1,5')
ARV.full_match('2.5')
ARV.no_match('x')
ARV.no_match('N/A')
ARV.no_match('teadmata arv')
ARV.partial_match('<=3', '3')
ARV.partial_match('~2', '2')

# Create pattern
ARVU_VAHEMIK = RegexElement(fr'{ARV}\s*(kuni|-)\s*{ARV}', 
                            description='Captures ranges of integer and decimal quantities.')
# Add tests
ARVU_VAHEMIK.full_match('1-2')
ARVU_VAHEMIK.full_match('100 kuni 150')
ARVU_VAHEMIK.full_match('1,5 - 2')
ARVU_VAHEMIK.full_match('2.5 kuni 3')
ARVU_VAHEMIK.no_match('x kuni y')
ARVU_VAHEMIK.no_match('N/A')
ARVU_VAHEMIK.no_match('teadmata arv')
ARVU_VAHEMIK.partial_match('~2-3', '2-3')

# Redefine KOGUS as ChoiceGroup
KOGUS = ChoiceGroup([ARVU_VAHEMIK, ARV], group_name='quantity', 
                    description='Captures integer and decimal quantities, including ranges.',
                    merge_positive_tests=True,
                    merge_negative_tests=True,
                    merge_extraction_tests=True)
# Add tests
KOGUS.full_match('2')
KOGUS.full_match('1,55')
KOGUS.full_match('3.5-4')
KOGUS.no_match('y')
KOGUS.no_match('??-??')
KOGUS.partial_match(' 3-4 ', '3-4')
KOGUS.partial_match('~2', '2')

Flags `merge_positive_tests`, `merge_negative_tests` and `merge_extraction_tests` can be used to guide which types of tests will be gathered from sub-expressions and carried over to the choice group expression. 
Note that there is no default strategy for deciding which tests should be carried over -- it is up for the user to guide the merging process.

In [16]:
# Check tests
KOGUS

Test group,passed,failed
positive examples,11,0
negative examples,6,0
extraction tests,5,0


In [17]:
# Check positive tests
KOGUS.evaluate_positive_examples()

,Example,Description,Status
0,1-2,(auto-merged test),+
1,100 kuni 150,(auto-merged test),+
2,"1,5 - 2",(auto-merged test),+
3,2.5 kuni 3,(auto-merged test),+
4,1,(auto-merged test),+
5,100,(auto-merged test),+
6,"1,5",(auto-merged test),+
7,2.5,(auto-merged test),+
8,2,,+
9,"1,55",,+


Description `(auto-merged test)` indicates that the test has been carried over from sub-expression tests.

### Output truncation

If patterns and examples grow long and fill up entire sections of the output, it is desirable to truncate output strings. While `RegexElement` does not apply output truncation by default, it provides class variables for switching on truncation and for modifying the maximum string length in the output.

Let's construct a final, long version of the KOOSTISOSA pattern:

In [18]:
# Create a pattern combining other patterns
KOOSTISOSA = RegexElement(fr'{KOGUS}\s*{YHIK}\s+{TOIDUAINED}', 
                          description='Captures ingredients together with quantities and units.')
# Add tests
KOOSTISOSA.full_match('1-2 supilusikat suhkrut')
KOOSTISOSA.full_match('2.5 sl võid')
KOOSTISOSA.full_match('4 kuni 5 teelusikat siirupit')
KOOSTISOSA.full_match('300g vahukoort')
KOOSTISOSA.no_match('10 tonni telliskive')
KOOSTISOSA.partial_match('100g vahukomme ka', {'quantity':'100', 'unit':'g', 'ingredient':'vahukomme'})
# Regular output
KOOSTISOSA

Test group,passed,failed
positive examples,4,0
negative examples,1,0
extraction tests,1,0


In [19]:
# Switch on truncation
RegexElement.TRUNCATE = True
# Set max string width
RegexElement.MAX_STRING_WIDTH = 100

In [20]:
# truncated output
KOOSTISOSA

Test group,passed,failed
positive examples,4,0
negative examples,1,0
extraction tests,1,0


Truncation is applied to examples and patterns longer than `MAX_STRING_WIDTH` in the `DataFrame` and HTML output, and in test error messages. 

### Finalizing the pattern

Once you've documented and covered a regex pattern with tests, use function `str(...)` to reveal the full regular expression string:

In [21]:
str(KOOSTISOSA)

'(?:(?P<quantity>(?:(?:([0-9]+[.,])?[0-9]+)\\s*(kuni|-)\\s*(?:([0-9]+[.,])?[0-9]+))|(?:([0-9]+[.,])?[0-9]+))\\s*(?P<unit>[Ss][Uu][Pp][Ii][Ll][Uu][Ss][Ii][Kk][Aa][Ss]|[Ss][Uu][Pp][Ii][Ll][Uu][Ss][Ii][Kk][Aa][Tt]|[Tt][Ee][Ee][Ll][Uu][Ss][Ii][Kk][Aa][Ss]|[Tt][Ee][Ee][Ll][Uu][Ss][Ii][Kk][Aa][Tt]|[Gg][Rr][Aa][Mm][Mm][Ii]|[Gg][Rr][Aa][Mm][Mm]|[Pp][Aa][Kk][Kk][Ii]|[Tt][Üü][Kk][Kk][Ii]|[Pp][Aa][Kk][Kk]|[Tt][Üü][Kk][Kk]|[Pp][Kk]|[Ss][Ll]|[Tt][Kk]|[Tt][Ll]|[Gg])\\s+(?P<ingredient>[Tt][Uu][Mm][Ee][Dd][Aa][Tt](?:\\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Vv][Aa][Ll][Gg][Ee][Tt](?:\\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Kk][Oo][Hh][Uu][Pp][Ii][Ii][Mm][Aa][Pp][Aa][Ss][Tt][Aa][Tt]|[Mm][Aa][Aa][Ss][Ii][Kk][Aa](?:\\s+)[Jj][Ää][Ää][Tt][Ii][Ss][Tt]|[Tt][Oo][Rr][Dd][Ii][Pp][Uu][Ll][Bb][Rr][Ii][Tt]|[Hh][Aa][Pp][Uu][Kk][Oo][Oo][Rr][Tt]|[Vv][Aa][Hh][Uu][Kk][Oo][Mm][Mm][Ee]|[Vv][Aa][Hh][Uu][Kk][Oo][Oo][Rr][Tt]|[Ss][Ii][Ii][Rr][Uu][Pp][Ii][Tt]|[Ss][Uu][Hh][Kk][Rr][Uu][Tt]|[Mm][Uu][Nn][Aa]|[Vv][Õõ][Ii][Dd]))

Use the class method `compile()` to convert the pattern into a `regex.Regex` object:

In [22]:
KOOSTISOSA.compile()

regex.Regex('(?:(?P<quantity>(?:(?:([0-9]+[.,])?[0-9]+)\\s*(kuni|-)\\s*(?:([0-9]+[.,])?[0-9]+))|(?:([0-9]+[.,])?[0-9]+))\\s*(?P<unit>[Ss][Uu][Pp][Ii][Ll][Uu][Ss][Ii][Kk][Aa][Ss]|[Ss][Uu][Pp][Ii][Ll][Uu][Ss][Ii][Kk][Aa][Tt]|[Tt][Ee][Ee][Ll][Uu][Ss][Ii][Kk][Aa][Ss]|[Tt][Ee][Ee][Ll][Uu][Ss][Ii][Kk][Aa][Tt]|[Gg][Rr][Aa][Mm][Mm][Ii]|[Gg][Rr][Aa][Mm][Mm]|[Pp][Aa][Kk][Kk][Ii]|[Tt][Üü][Kk][Kk][Ii]|[Pp][Aa][Kk][Kk]|[Tt][Üü][Kk][Kk]|[Pp][Kk]|[Ss][Ll]|[Tt][Kk]|[Tt][Ll]|[Gg])\\s+(?P<ingredient>[Tt][Uu][Mm][Ee][Dd][Aa][Tt](?:\\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Vv][Aa][Ll][Gg][Ee][Tt](?:\\s+)[Šš][Oo][Kk][Oo][Ll][Aa][Aa][Dd][Ii]|[Kk][Oo][Hh][Uu][Pp][Ii][Ii][Mm][Aa][Pp][Aa][Ss][Tt][Aa][Tt]|[Mm][Aa][Aa][Ss][Ii][Kk][Aa](?:\\s+)[Jj][Ää][Ää][Tt][Ii][Ss][Tt]|[Tt][Oo][Rr][Dd][Ii][Pp][Uu][Ll][Bb][Rr][Ii][Tt]|[Hh][Aa][Pp][Uu][Kk][Oo][Oo][Rr][Tt]|[Vv][Aa][Hh][Uu][Kk][Oo][Mm][Mm][Ee]|[Vv][Aa][Hh][Uu][Kk][Oo][Oo][Rr][Tt]|[Ss][Ii][Ii][Rr][Uu][Pp][Ii][Tt]|[Ss][Uu][Hh][Kk][Rr][Uu][Tt]|[Mm][Uu][Nn][Aa]|[Vv][Õ

You can also pass [flags](https://docs.python.org/3/library/re.html#flags), such as `regex.IGNORECASE` and `regex.DOTALL`, to the `compile()` method.

### Limitations

The encapsulation provided by `RegexElement` makes it much safer to specify what should be matched by the regular expression, but it still has limitations. 
First, one cannot specify additional consistency constraints inside the hierarchical definition nor aggregate the contents of capture groups. 
If you need such features, use grammar rules instead. 
Second, self-overlapping can cause subtle errors. This is particularly true in case of string replacement. One way to diagnose is to compare `regex.sub(..., count=-1)` and several invocations of `regex.sub.(..., count=1)` to see if there are some differences.

### Using the pattern with RegexTagger

After creating a pattern, you can use it in a `RegexTagger`. 
Here, we give a brief example on how to do that, without going into details. 
Please see other tutorials in this folder for detailed explanations on how to construct rules and how to create rule taggers. 

In [23]:
from estnltk import Text
from estnltk.taggers import RegexTagger
from estnltk.taggers.system.rule_taggers import Ruleset
from estnltk.taggers.system.rule_taggers import StaticExtractionRule

# Make rule set for RegexTagger
rule_list = [StaticExtractionRule(pattern=KOOSTISOSA.compile())]
ruleset = Ruleset()
ruleset.add_rules(rule_list)

# Use a decorator to rewrite capture group values to annotations
def decorator(layer, base_span, annotation):
    annotation['quantity'] = annotation['match'].group('quantity')
    annotation['unit'] = annotation['match'].group('unit')
    annotation['ingredient'] = annotation['match'].group('ingredient')
    return annotation

# Create RegexTagger
regex_tagger = RegexTagger(ruleset=ruleset, 
                           output_layer='food_ingredients',
                           output_attributes=['quantity', 'unit', 'ingredient'], 
                           decorator=decorator)

In [24]:
# Example for analysis
example_text = """
1 pakk tordipulbrit 
4 tk muna 
400 g kohupiimapastat
4 sl suhkrut
200 g hapukoort
100 g võid
"""

# Apply the tagger
text = Text(example_text)
regex_tagger.tag(text)

# Check the results
text['food_ingredients']

Layer(name='food_ingredients', attributes=('quantity', 'unit', 'ingredient'), spans=SL[Span('1 pakk tordipulbrit', [{'quantity': '1', 'unit': 'pakk', 'ingredient': 'tordipulbrit'}]),
Span('4 tk muna', [{'quantity': '4', 'unit': 'tk', 'ingredient': 'muna'}]),
Span('400 g kohupiimapastat', [{'quantity': '400', 'unit': 'g', 'ingredient': 'kohupiimapastat'}]),
Span('4 sl suhkrut', [{'quantity': '4', 'unit': 'sl', 'ingredient': 'suhkrut'}]),
Span('200 g hapukoort', [{'quantity': '200', 'unit': 'g', 'ingredient': 'hapukoort'}]),
Span('100 g võid', [{'quantity': '100', 'unit': 'g', 'ingredient': 'võid'}])])

---